In [14]:
import pandas as pd
import numpy as np

# Read Fixed with File and Parse Columns 
## File Notes: 
* File contains Header name 
* First 108 rows contains another dataset 
* File contains empty footer row 
* Every 48th row is a new page with 8 header rows 
* Job ID does not repete for every line 

In [3]:
#Read Fixed width file, drop first 109 rows and last row from file 
data = pd.read_fwf('C:\Downloads\ACR_History_db.txt',header=1, skiprows=108, skipfooter=1, infer_nrows=10) 
data.columns

Index(['JOB ID', 'CYCLE#', 'RUN#', 'CODE', 'STATUS', 'RUN DATE', 'RUN TIME',
       'DATE      TIME   NO.', 'DEFN', 'HISTORY', 'NAME'],
      dtype='object')

# Remove unused data
* Drop First and Last row
* Reset Index
* Drop Unused Columns 


In [4]:
data.drop(data.index[[0]], inplace=True) # Drop First Record
# Remove unused columns 
data.drop(['NAME', 'HISTORY', 'DEFN', 'CYCLE#', 'STATUS', 'DATE      TIME   NO.', 'RUN TIME'], axis=1, inplace = True) 
data = data.reset_index(drop=True) # Reindex 
data.tail(15)


,JOB ID,RUN#,CODE,RUN DATE
22052,NaN,000,0000,18/07/12
22053,NaN,000,0000,18/07/13
22054,NaN,000,0000,18/07/16
22055,NaN,000,0000,18/07/17
22056,1ACR/S R40V7M00,NaN,NaN,UNITEC
22057,0DATE: 19/10/16,NaN,NaN,INF
22058,0TIME: 20:25:40,NaN,NaN,HIS
22059,NaN,NaN,NaN,DSN
22060,0,NaN,NaN,SUM
22061,0,NaN,RETURN,NaN


# Cleanup Dataset
* Drop non data rows
* Fill in Missing Job Names

In [11]:
#create list of indexs to drop 
count = 0 # Create counter
droprow = [] #Create blank List
#for row in range(len(data.index)-1): # for ever row the len the dataframe -1 (len does not account for 0 index value)
for row in range(len(data)):
    if  48 <= count < 56:
            count = count + 1
            droprow.append(False)
            #droprow.append(row)
            continue
    elif count >= 56:    
        count = 1 
        droprow.append(True)
    else:
        droprow.append(True)
        count = count + 1 

#data.drop(data.index[droprow], inplace=True) # drop rows of the index 
data = data[droprow].copy() #drop rows based on boolean index

# fill in missing job names by propogating last valid value forward 
data['JOB ID'].fillna(method= 'pad', inplace=True) # propogate Job IDs to next row if empty 
data.tail(5)

,JOB ID,RUN#,CODE,RUN DATE
22054,PETBI156STEP888,000,0000,18/07/16
22055,PETBI156STEP888,000,0000,18/07/17
22064,PETBI156STEP888,000,0000,18/07/18
22065,PETBI156STEP888,000,0000,18/07/19
22066,PETBI156STEP888,000,0000,18/07/20


# Filter Data
* Select only records with error code
* Remove job 'PBTCPU10STEP888'-  Job renamed to PAPBI010
* Remove job 'PBTIBM40STEP888'-  Job Not in Lynns ACR Doc
* Remove job 'PCMDV010STEP888'-  Job Not in Lynns ACR Doc
* Sort by status, rundate, job id

### old code
%%timeit -n1 -r25 
qr_old = data.query('CODE !="0000"').copy() # Removed - jobs that have not flagged and error since 2016
#qr = qr[qr['JOB ID'] != 'PBTCPU10STEP888'] # Removed - Job renamed to PAPBI010
qr_old = qr.replace('PBTCPU10STEP888', 'PAPBI010STEP888' ) # Rename job - Job renamed to PAPBI010
qr_old = qr[qr['JOB ID'] != 'PBTIBM40STEP888'] # Removed - Job Not in Lynn's ACR Doc
qr_old = qr[qr['JOB ID'] != 'PBTIBM40STEP888'] # Removed - Job Not in Lynn's ACR Doc


qr_old.sort_values(by= ['RUN DATE','JOB ID'], ascending=False,  inplace = True)
#qr.to_excel('C:\Downloads\ACR History DB Cleaned.xlsx') # Write full dataset to excel file
qr_old

In [24]:
#%%timeit -n1 -r25 
qr = data[np.logical_and(data['CODE'] != "0000", 
                         np.isin(data['JOB ID'], ['PBTIBM40STEP888', 'PBTIBM40STEP888'], invert = True))]
qr = qr.replace('PBTCPU10STEP888', 'PAPBI010STEP888' )
qr.sort_values(by= ['RUN DATE','JOB ID'], ascending=False,  inplace = True)
qr
#qr.to_excel('C:\Downloads\ACR History DB Cleaned.xlsx') # Write full dataset to excel file qr_old

,JOB ID,RUN#,CODE,RUN DATE
17963,PELRG140STEP010,000,1030,96/04/04
17884,PELRG140STEP010,000,1030,95/12/26
17821,PELRG140STEP010,000,1030,95/10/09
17736,PELRG140STEP010,000,1030,95/07/03
11846,PATUN210STEP888,000,1018,19/09/06
...,...,...,...,...
13985,PAPBI010STEP888,000,1021,05/12/01
13861,PAPBI010STEP888,000,1021,05/07/01
3,MDBPSYNCSTEP888,000,1058,04/12/02
17329,PCMDV010STEP888,000,1047,04/08/29


# Group by and Generate final results table
* Group by Job ID, Code with max Run date
* Group by Job ID, Code with count
* Merge results into one table

In [6]:
maxdate = qr.groupby(['JOB ID', 'CODE']).max() # Get max date dataset 
maxcount = qr.groupby(['JOB ID', 'CODE']).count() # get count of dataset 
#join 2 datasets in Job ID and Code and only use the Run column and Run date columns 
final = pd.merge(maxcount['RUN#'].astype('str').to_frame(), maxdate['RUN DATE'].to_frame(), on = ['JOB ID', 'CODE']) 
final = final.rename ( columns = {'RUN DATE': " Max Run date ", "RUN#" : "Count"}) # Rename Column
final

Count  Max Run date 
JOB ID          CODE                     
MDBPSYNCSTEP888 1058     1       04/12/02
PACBI020STEP888 1005     1       18/12/05
PACXT600STEP888 1023     2       19/03/26
PAPBI010STEP888 1003     3       19/07/05
                1021     7       06/02/23
PATUN210STEP888 1018     1       19/09/06
PATUN300STEP888 1018     1       19/08/09
PELRG140STEP010 1030     4       96/04/04
PETBI127STEP888 1015   365       18/03/29
PETBI150STEP888 1012     1       12/09/10

# Appendix 

# Get Max, Min and Count by jobid 

In [7]:
try:
    fjobid = data.query('`JOB ID` ==  "PBTCPU10STEP888"').copy()
    Count = fjobid.filter(['JOB ID', 'CODE', 'RUN DATE']).groupby(['JOB ID', 'CODE']).count()

    maxval = fjobid.filter(['JOB ID', 'CODE', 'RUN DATE']).groupby(['JOB ID', 'CODE']).max()
    minval = fjobid.filter(['JOB ID', 'CODE', 'RUN DATE']).groupby(['JOB ID', 'CODE']).min()

    Count = Count.rename ( columns = {'RUN DATE': "Count "})
    maxval = maxval.rename( columns = {'RUN DATE': "Max Date "})
    minval = minval.rename( columns = {'RUN DATE': "Min Date"})


    maxmin = pd.merge(maxval, minval, on = ['JOB ID', 'CODE'])
    maxmincount =pd.merge(maxmin, Count,on = ['JOB ID', 'CODE'] )
    display(maxmincount)
except:
    print('job not found')

Max Date   Min Date  Count 
JOB ID          CODE                            
PBTCPU10STEP888 0000  06/03/03  05/06/08     182
                1021  06/02/23  05/07/01       7

In [8]:
## Other queries 

In [9]:
##qr.groupby(['JOB ID', 'CODE']).count().sort_values(['JOB ID', 'CODE'])
##qr.groupby(['JOB ID', 'CODE']).max().sort_values(['JOB ID', 'CODE'])